# Distribuicões de variáveis dentro das classes

In [1]:
# Importar as Bibliotecas
import collections

import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

from sklearn import datasets
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss


In [ ]:
df_iris = datasets.load_iris()
name_columns = df_iris.feature_names
des_target = df_iris.target_names

df = pd.DataFrame(data=df_iris.data, columns=name_columns)
df['target'] = df_iris.target

In [ ]:
print(collections.Counter(df_iris.target))

In [ ]:
des_target

In [ ]:
df.head()

In [ ]:
for feature in name_columns:
    print(f"Feature analizada: {feature}")
    sn.catplot(x="target", y=feature, kind="box", data=df)
    plt.show()

## Escor Z

Às vezes, os dados são padronizados primeiro (por exemplo, para um escore Z com média zero e variância da unidade) de modo que a detecção de outlier possa ser realizada usando valores de corte de escore Z padrão. Isso é uma conveniência e não é obrigatório em geral, e faremos os cálculos na escala original dos dados aqui para tornar as coisas mais claras. Podemos calcular a média e o desvio padrão de uma determinada amostra e, em seguida, calcular o cut-off para identificar outliers como mais de 2 desvios padrão da média.

In [ ]:
for variavel in name_columns:
    df_defaut = []
    for value, target in enumerate(des_target):
        dados = df[df["target"]==value]
        print(f"Classe: {des_target[value]} - variavel: {variavel}")

        data_mean, data_std = dados[variavel].mean(), dados[variavel].std()
        data_min, data_max = dados[variavel].min(), dados[variavel].max()
        print("Real Min: %.3f Real Max: %.3f" %  (data_min, data_max))

        cut_off = data_std * 2.5
        lower, upper = data_mean - cut_off, data_mean + cut_off
        print("Limit Min: %.3f Limit Max: %.3f" %  (lower, upper))


        dataset = dados[dados[variavel] >= lower]
        if len(dataset) == 0:
            dataset = dados[dados[variavel] <= upper]
        else:
            dataset = dataset[dataset[variavel] <= upper]


        df_defaut.append(dataset)

        outliers = pd.concat([dados[dados[variavel] < lower], dados[dados[variavel] > upper]])
        print("Identfied outliers: %d \n" % len(outliers))

    df = pd.concat(df_defaut)


In [ ]:
print(f"shape : {df.shape}")

In [ ]:
print(collections.Counter(df.target))

## Lidando com Dados Desbalanceados

Após essa introdução e apresentação dos problemas causados pelos Dados Desbalanceados, vamos agora estudar algumas das formas mais comuns que existem para resolver esse empecilho.

### Reestruturação dos Dados
Uma forma de tirar o viés causado pela diferença de proporção das categorias consiste em manipular a quantidade de dados que são efetivamente utilizados pelo modelo de Machine Learning, tentando igualar o número de observações entre as classes.

### Undersampling
Esse método consiste em reduzir o número de observações da classe majoritária para diminuir a diferença entre as categorias.

* **Random Undersampler:** que consiste na retirada aleatória de dados da classe majoritária (o que acarreta em uma perda grave de informação)

* **NearMiss:** Refere-se a uma coleção de métodos de sub-amostragem que selecionam exemplos com base na distância dos exemplos de classes majoritárias aos exemplos de classes minoritárias.

ref: https://python-data-science.readthedocs.io/en/latest/classimbalance.html


In [ ]:
# Gera dataset e labels
y = df["target"].to_numpy()
X = df
X = X.drop("target", axis=1)
X = X.to_numpy()

In [ ]:
# Random Undersampler
rus = RandomUnderSampler(random_state = 32)
X_rus_res, y_rus_res = rus.fit_resample(X, y)
print(collections.Counter(y_rus_res))


In [ ]:
# NearMiss
nm = NearMiss(version=1)
X_nm_res, y_nm_res = nm.fit_resample(X, y)
print(collections.Counter(y_nm_res))

### Oversampling
Ao contrário do Undersampling, o Oversampling consiste em criar sinteticamente novas observações da classe minoritária, com o objetivo de igualar a proporção das categorias.

* **SMOTE (Synthetic Minority Oversampling Technique):** Um problema com a classificação desequilibrada é que há poucos exemplos da classe da minoria para que um modelo aprenda efetivamente o limite de decisão. Uma melhoria na duplicação de exemplos da classe minoritária é sintetizar novos exemplos da classe minoritária. Este é um tipo de aumento de dados para dados tabulares e pode ser muito eficaz.

In [ ]:
overSampling = SMOTE()
X, y = overSampling.fit_resample(X, y)
print(collections.Counter(y))